In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import numpy as nb
from config import *
from utils import *
import json
import re
import datetime as dt

In [3]:
scenario_file_name = "generated_scenarios_20250922_1207"

In [4]:
output_file = path_results + scenario_file_name + ".jsonl"

In [5]:
df_crh= pd.read_csv(path_results + "crh_"+scenario_file_name + ".csv",index_col=0)

In [14]:
re.search(r"```json(.*?)```", df_crh.crh.iloc[0], re.DOTALL)

In [213]:
limit = 50
with open(output_file, 'a+', encoding="utf-8") as outfile:

    for i in range(0,len(df_crh)) :

        if  len(df_crh.crh[i])<500 :

            continue
        # annotations = json.loads(df_crh.crh[i].split('"formulations":')[1][:-3])
        case_management_type = df_crh.case_management_type[i]
        case_management_type_description = df_crh.case_management_type_description[i]

        diagnostics = df_crh.crh[i].split('"formulations":')[1][:-3].replace('\n ',"").split("informations")[0][:-5].split('"diagnostics":')[1]

        text_crh = df_crh.crh[i].split('"formulations":')[0].split('"CR":')[1][3:-5]

        codes = []
        #Regular expressions 
        p = re.compile("^[A-Z][0-9]{2}")
        PCID = "Diagnostic principal :\n"
        SICD = "Diagnotics associés :\n"
        CMT = "Motif de recours au soin (code en Z du chapitre XXI):\n"

        if case_management_type == "DP":
            CMT+= "- aucun, le motif de recours au soin est le diagnostic principal\n"
        else :
            CMT+= "- " + case_management_type_description + "(" + case_management_type + ")\n"

        for x in re.sub('"',"", diagnostics).split("],"):
            diag =  re.sub("\[|\]|}|{","",x).replace("  ","")
            extract = re.findall(r'\((.*?)\)',diag)
            if extract:
                for code in extract: 
                    if p.match(code):
                        codes +=[code]
                        if(code == df_crh.icd_primary_code.loc[i]):
                            PCID += "- " + diag + "\n"
                        else :
                            SICD += "- " + diag + "\n"
        
                    
            
        data = {
        "messages": [
            {
                "role": "system",
                "content": """Vous êtes un modèle de langage en française spécialisé dans le codage des diagnostics selon la classification internationale des maladies version 10 (CIM-10) pour les résumés standardisés de sortie du programme de médicalisation des systèmes d'information français (PMSI). A partir des comptes rendus d'hospitalisation vous donnerez  les codes diagnostics CIM-10 que l'on peut retenir pour le séjours en distiguant  :
- Motifs de recours (code en Z du chapitre XXI). Ces codes ne sont utilisables que dans certaines circonstances. Lorsqu'il n'est pas nécessaire vous d'utiliser un code en Z vous votre réponse pour cette catégorie sera : aucun, le motif de recours au soin est le diagnostic principal.
- Diagnostic principal 
- Diagnostics associés """},

            {
                "role": "user",
                "content": "Générez le codage CIM-10 du résumé strandisé de sortie PMSI à partir du compte rendu d'hospitalisation suivant : " + text_crh
            },
            {
                "role": "assistant",
                "content": "Codes CIM 10 retenus pour le résumé strandisé de sortie PMSI : " + CMT+PCID+SICD
            }
        ]
        }
        json.dump(data, outfile, separators=(',', ':'),  ensure_ascii=False)
        outfile.write('\n')


        if limit is not None:
            if i > limit:
                break

In [220]:
print(CMT+PCID+SICD)

Motif de recours au soin (code en Z du chapitre XXI):
- aucun, le motif de recours au soin est le diagnostic principal
Diagnostic principal :
- Travail et accouchement compliqués d'une anomalie du rythme cardiaque du foetus (O680):anomalies du rythme cardiaque fœtal, décélérations variables du RCF, tracé de type II, signes de détresse fœtale chronique, anomalies du RCF persistantes 
Diagnotics associés :
-  Centres médicaux non disponibles et non accessibles (Z753):difficultés d’accès aux soins, suivi obstétrical irrégulier, éloignement géographique, précarité sociale 
-  Maladies à protozoaires compliquant la grossesse, l'accouchement et la puerpéralité (O986):paludisme traité en cours de grossesse, antécédent de paludisme à *Plasmodium falciparum*, sérologie paludéenne positive au 2ᵉ trimestre, contexte infectieux antérieur (paludisme) 
-  Rupture d'une suture de césarienne (O900):risque de désunion de cicatrice de césarienne// Mentionné indirectement dans la surveillance post-op 
- 

In [214]:
from mistralai import Mistral
import os


client = Mistral(api_key=api_key)

file_1 = client.files.upload(file={
    "file_name": scenario_file_name + ".jsonl",
    "content": open(output_file, "rb"),
})


In [217]:
scenario_file_name = "generated_scenarios_20250922_1208"
output_file = path_results + scenario_file_name + ".jsonl"

In [218]:
file_2 = client.files.upload(file={
    "file_name": scenario_file_name + ".jsonl",
    "content": open(output_file, "rb"),
})


In [219]:
# create a fine-tuning job
created_jobs = client.fine_tuning.jobs.create(
    model="open-mistral-7b", 
    training_files=[{"file_id": file_1.id, "weight": 1}],
    validation_files=[file_2.id], 
    hyperparameters={
        "training_steps": 10,
        "learning_rate":0.0001
    },
    auto_start=False
)

# start a fine-tuning job
client.fine_tuning.jobs.start(job_id = created_jobs.id)

created_jobs

SDKError: API error occurred: Status 400
{"detail": "Job has status QUEUED, cannot start unless job has status VALIDATED."}

In [38]:
import ast

In [45]:
#job_name="sample_10000_20250924"
job_name="sample_5000_20250919"
file_name_finetune = "train"
#file_name_finetune = "val"
nb_files = 5

df_res_final = pd.DataFrame()
#nb_examples = 100
nb_examples = None
j = 0
for i in range(nb_files):
    file_path = os.path.join(path_results + job_name +"/batch_" +  str(i) + ".json")
    if os.path.exists(file_path):
        results = []
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                results.append(json.loads(line.strip()))
                j+=1
                if nb_examples is not None:
                    if j> nb_examples:
                        break
        
        prep_dict=[]
        
        for result_item in results:
                # Check if the item has a response and extract it
                if "response" in result_item and result_item["response"] and "body" in result_item["response"] and "choices" in result_item["response"]["body"]:
                    response = result_item["response"]["body"]["choices"][0]["message"]["content"]
                    clinical_report,response_diagnosis,response_structured_data = extract_generations_annotations(response)
                    prep_dict.append({"bacth":i,"num_in_":result_item['custom_id'],"clinical_report":clinical_report,"response_diagnosis":response_diagnosis,"response_structured_data":response_structured_data })
        
        df_scenarios_save = pd.read_csv(path_results + job_name +"/batch_" +  str(i) + ".csv",index_col=0)
        df_res_tmp = pd.DataFrame(prep_dict)
        df_res_tmp["num_in_"]=  df_res_tmp["num_in_"].astype(int)
        df_res_tmp = df_scenarios_save.merge(df_res_tmp,right_on="num_in_",left_index=True)
        now = dt.datetime.now()
        now_string = now.strftime("%Y%m%d%H%M%S%f")
        df_res_tmp = df_res_tmp.assign(encounter_id = now_string + df_res_tmp.bacth.apply(str) + df_res_tmp.num_in_.apply(str))
        df_res_tmp.encounter_id = df_res_tmp.encounter_id.str.pad(width=10, side='left', fillchar='0')
        df_res_final = pd.concat([df_res_final,df_res_tmp])    

In [48]:
df_res_final = df_res_final[df_res_final.clinical_report.notna()]

In [49]:
df_res_final["icd_primary_pred"], df_res_final["icd_secondary_pred"],df_res_final["icd_coding_text"],df_res_final["icd_coding_list"] = zip(*df_res_final.apply(get_icd_coding_target,axis=1))

In [50]:
cols = ['encounter_id','icd_primary_code', 'case_management_type','icd_parent_code','icd_primary_pred','icd_secondary_pred', 'icd_coding_text', 'icd_coding_list','clinical_report']

In [44]:
df_res_final_sav = df_res_final.copy()

In [51]:
df_res_final_sav = pd.concat([df_res_final_sav[cols], df_res_final.iloc[:(10000-len(df_res_final_sav)),:][cols]],axis=0)

In [52]:
file_name_finetune = "train"
df_res_final_sav.to_csv(path_results + job_name +"/"+file_name_finetune+".csv")
print(path_results + job_name +"/"+file_name_finetune+".csv")

P:/Commun/Sources_donnees_complementaires/generation/recode_scenario/sample_5000_20250919/train.csv


In [53]:
file_name_finetune = "val"
df_res_final.iloc[2000:5000,:][cols].to_csv(path_results + job_name +"/"+file_name_finetune+".csv")
print(path_results + job_name +"/"+file_name_finetune+".csv")

P:/Commun/Sources_donnees_complementaires/generation/recode_scenario/sample_5000_20250919/val.csv


In [116]:
df_res_final[["encounter_id","clinical_report","icd_coding_text","icd_coding_list"]].to_csv(path_results + job_name +"/"+file_name_finetune+".csv")
print(path_results + job_name +"/"+file_name_finetune+".csv")

P:/Commun/Sources_donnees_complementaires/generation/recode_scenario/sample_10000_20250924/train.csv


: 

In [48]:
from config import *
api_url = "https://api.mistral.ai/v1/chat"
model = "mistral-large-latest"
client = Mistral(api_key=api_key)

In [59]:
i=0
job_name="generated_scenarios_10000_20250924"
file_name = "20f84db0-d578-48b3-b8a2-80fd4a957c29" 
download_file(client, file_name, path_results + job_name +"/batch_" +  str(i) + ".json" )

Downloaded file to P:/Commun/Sources_donnees_complementaires/generation/recode_scenario/generated_scenarios_10000_20250924/batch_0.json
